In [1]:
!pip install --upgrade kiteconnect
!pip install python-dotenv
!pip install python-dateutil

Requirement already up-to-date: kiteconnect in c:\users\nisha\miniconda3\lib\site-packages (3.9.4)


In [2]:
import logging
import os
from kiteconnect import KiteConnect
import logging
from kiteconnect import KiteTicker

logging.basicConfig(level=logging.DEBUG)
from dotenv import load_dotenv
load_dotenv()
import bs_threading
import importlib
import time
from datetime import datetime, timedelta
import dateutil
import pandas as pd
importlib.reload(bs_threading)
from bs_threading import bs_threadify, bs_make_throttle_ready_func


In [4]:
KITE_CONNECT_API_KEY = os.getenv("KITE_CONNECT_API_KEY")
KITE_CONNECT_API_SECRET = os.getenv("KITE_CONNECT_API_SECRET")

In [4]:

logging.basicConfig(level=logging.DEBUG)

kite = KiteConnect(api_key=KITE_CONNECT_API_KEY)



### Get the url based on your api key, which will have request token, this is in order to verify the app itself(bsstonks) is valid and it'll get redirected to the callback url

In [5]:
kite.login_url()

'https://kite.trade/connect/login?api_key=7e6nr4sfi674uleu&v=3'

### Given we have request token and redirected to our callback with it, our app is valid, but is the app in the hands/server of the right person or not? only API secret can verify that. 
### We'll pass that api secret with the request token to create a session which give us Access_token, to the rightful person that we are

In [6]:
# Redirect the user to the login url obtained
# from kite.login_url(), and receive the request_token
# from the registered redirect url after the login flow.
# Once you have the request_token, obtain the access_token
# as follows.

kite_session_data = kite.generate_session("puaTPBINCesLweOkxnKMKMWmdtIeNrCs", api_secret=KITE_CONNECT_API_SECRET)
print("THE TOKEN:",kite_session_data["access_token"])
kite.set_access_token(kite_session_data["access_token"])

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "POST /session/token HTTP/1.1" 200 None


THE TOKEN: RvEnBhwwtPktS1EinG3PNlPddlouLKNe


In [5]:
kite.set_access_token("RvEnBhwwtPktS1EinG3PNlPddlouLKNe")

In [10]:
kite.orders()
kite.holdings()

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /orders HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /portfolio/holdings HTTP/1.1" 200 None


[{'tradingsymbol': 'BHEL',
  'exchange': 'NSE',
  'instrument_token': 112129,
  'isin': 'INE257A01026',
  'product': 'CNC',
  'price': 0,
  'quantity': 250,
  'used_quantity': 0,
  't1_quantity': 0,
  'realised_quantity': 250,
  'authorised_quantity': 0,
  'authorised_date': '2021-06-29 00:00:00',
  'opening_quantity': 250,
  'collateral_quantity': 0,
  'collateral_type': '',
  'discrepancy': False,
  'average_price': 67.9,
  'last_price': 65.7,
  'close_price': 66.3,
  'pnl': -550.0000000000007,
  'day_change': -0.5999999999999943,
  'day_change_percentage': -0.9049773755656023},
 {'tradingsymbol': 'MANGALAM',
  'exchange': 'NSE',
  'instrument_token': 3025153,
  'isin': 'INE584F01014',
  'product': 'CNC',
  'price': 0,
  'quantity': 1,
  'used_quantity': 0,
  't1_quantity': 0,
  'realised_quantity': 1,
  'authorised_quantity': 0,
  'authorised_date': '2021-06-29 00:00:00',
  'opening_quantity': 1,
  'collateral_quantity': 0,
  'collateral_type': '',
  'discrepancy': False,
  'average

In [6]:
# Initialise
kws = KiteTicker(KITE_CONNECT_API_KEY, "RvEnBhwwtPktS1EinG3PNlPddlouLKNe")

def on_ticks(ws, ticks):
    # Callback to receive ticks.
    logging.debug("Ticks: {}".format(ticks))

def on_connect(ws, response):
    # Callback on successful connect.
    # Subscribe to a list of instrument_tokens (RELIANCE and ACC here).
    ws.subscribe([738561, 5633])

    # Set RELIANCE to tick in `full` mode.
    ws.set_mode(ws.MODE_FULL, [738561])

def on_close(ws, code, reason):
    # On connection close stop the event loop.
    # Reconnection will not happen after executing `ws.stop()`
    ws.stop()

# Assign the callbacks.
kws.on_ticks = on_ticks
kws.on_connect = on_connect
kws.on_close = on_close

# Infinite loop on the main thread. Nothing after this will run.
# You have to use the pre-defined callbacks to manage subscriptions.
kws.connect()

ReactorNotRestartable: 